In [1]:
import pandas as pd
import numpy as np
import time
import iddModel.learnUsingNumpy as idoctor
import multiprocessing
import parmap 
import math
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

In [7]:
tuningSetAgeRate = [[0.1, 0.2, 0.9],[26,28,27],[40,40,40]]
tuningSetRetireRate = [1.4,35,0.2]
%time npCostData = idoctor.getCost([tuningSetAgeRate, tuningSetRetireRate])
pd.DataFrame(npCostData)

CPU times: user 18.9 ms, sys: 2.63 ms, total: 21.5 ms
Wall time: 19.2 ms


,0,1,2,3,4,5,6,7,8,9,10,11
0,9363,7496,7192,158,4060,1423,0.0235171,0.075714,0.0463351,0.039638,0.0728811,0.0330471
1,5614,4018,521,35,425,1535,0.0235171,0.0406263,0.0109217,0.0107113,0.00914613,0.0136699
2,9455,6008,6718,37,2715,519,0.0241584,0.0574154,0.03852,0.0403067,0.0571661,0.0226211
3,5861,3130,1027,174,151,1681,0.0241584,0.0300901,0.00638348,0.00821989,0.00613998,0.0148116
4,8507,4448,5359,298,1408,406,0.0217444,0.0397038,0.0254623,0.0370648,0.039307,0.0105387
5,5480,2282,828,285,238,1847,0.0217444,0.0203947,0.00764586,0.00573617,0.00163245,0.0163642
6,8591,3174,4666,357,275,1383,0.0230566,0.0251675,0.0170355,0.0368999,0.0276155,0.000744157
7,5841,1636,1091,479,609,2026,0.0230566,0.0127615,0.00558802,0.00373219,0.00192564,0.0176897
8,8660,1681,4360,818,994,2443,0.023607,0.00922037,0.0128593,0.0401576,0.0151014,0.00957248
9,6093,631,1590,651,1060,2161,0.023607,0.00199955,0.000940126,0.00210521,0.00607971,0.018573


In [3]:
def runGetCost(tuningValueList):
    resultData = []
    
    for value in tuningValueList:
        resultData.append([value ,idoctor.getCost([[[value[0], value[1], value[2]],[26,28,27],[40,40,40]], [value[3],30,value[4]]])])
        
        
    return resultData

In [33]:
def startGetCost():
    tAge1 = np.arange(0.05, 1, 0.05) # 연령분포모델1 변수 / 범위 0~1
    tAge2 = np.arange(0.05, 1, 0.05) # 연령분포모델2 변수 / 범위 0~1
    tAge3 = np.arange(0.1, 1, 0.1) # 연령분포모델3 변수 / 범위 0~1
    tRetire1 = np.arange(1.1, 3, 0.1) # 은퇴율모델 변수1 / 범위 1 이상
    tRetire2 = np.arange(0.1, 10, 0.1) # 은퇴율모델 변수3 / 범위 0 이상
    
    dataSize = len(tAge1)*len(tAge2)*len(tAge3)*len(tRetire1)*len(tRetire2)
    timeMinute = dataSize*20/1000/60/22
    curentData = 0
    resultData = []
    
    num_cores = 30

    now = time.localtime()
    print("│ Start Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    print("│ expect running time : " + str(round(timeMinute)) + "min")
    print("│ Data Size = " + str(dataSize))
    print("└───────────────────────")
    print('│ '+str(round(curentData/dataSize*100,2))+'%')
    
    tuningList = []
    
    for i in tAge1:
        for j in tAge2:
            for k in tAge3:
                for m in tRetire1:
                    for n in tRetire2:
                        tuningList.append([i,j,k,m,n])

    
    splitedTuningList = np.array_split(tuningList, num_cores) 
    splitedTuningList = [i.tolist() for i in splitedTuningList] 
    
    result = parmap.map(runGetCost, splitedTuningList, pm_pbar=True, pm_processes=num_cores)
    
    print("| End Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    return result

In [34]:
%time liCostData = startGetCost()
npCostData = np.array(liCostData)

np.shape(npCostData)

│ Start Time : 2020/11/10 18:24:52
│ expect running time : 93min
│ Data Size = 6111369
└───────────────────────
│ 0.0%


100%|██████████| 30/30 [1:39:23<00:00, 198.77s/it]   


| End Time : 2020/11/10 18:24:52
CPU times: user 2min 21s, sys: 1min 17s, total: 3min 38s
Wall time: 1h 39min 52s


(30,)

In [35]:
len(liCostData[0])
npCostData = np.array(liCostData)

In [ ]:
value = [0.4, 0.8, 0.4, 40, 1.09, 30, 1.9]

data = idoctor.getCost([[[value[0], value[1], value[2]],[26,28,27],[value[3],value[3],value[3]]], [value[4],value[5],value[6]]])
data

In [42]:
minManCost = (np.concatenate((npCostData[0][0][1][0],npCostData[0][0][1][2],npCostData[0][0][1][4],npCostData[0][0][1][6],npCostData[0][0][1][8],npCostData[0][0][1][10]))).reshape(6,12).sum(axis=0)
minWomanCost = (np.concatenate((npCostData[0][0][1][1],npCostData[0][0][1][3],npCostData[0][0][1][5],npCostData[0][0][1][7],npCostData[0][0][1][9],npCostData[0][0][1][11]))).reshape(6,12).sum(axis=0)
minTuningValue = npCostData[0][0][0]
minTuningValue

[0.05, 0.05, 0.1, 1.1, 0.1]

In [51]:
minManCost = (np.concatenate((npCostData[0][0][1][0],npCostData[0][0][1][2],npCostData[0][0][1][4],npCostData[0][0][1][6],npCostData[0][0][1][8],npCostData[0][0][1][10]))).reshape(6,12).sum(axis=0)
minWomanCost = (np.concatenate((npCostData[0][0][1][1],npCostData[0][0][1][3],npCostData[0][0][1][5],npCostData[0][0][1][7],npCostData[0][0][1][9],npCostData[0][0][1][11]))).reshape(6,12).sum(axis=0)
minManTuningValue = npCostData[0][0][0]
minWomanTuningValue = npCostData[0][0][0]
count = 0
for data1 in npCostData:
    for data2 in data1:   
        count += 1
        npManCost = (np.concatenate((data2[1][0],data2[1][2],data2[1][4],data2[1][6],data2[1][8],data2[1][10]))).reshape(6,12).sum(axis=0)
        npWomanCost = (np.concatenate((data2[1][1],data2[1][3],data2[1][5],data2[1][7],data2[1][9],data2[1][11]))).reshape(6,12).sum(axis=0)

        manCostCount = 0
        womanCostCount = 0
        
        for i in range(12):
            if npManCost[i]<=minManCost[i]:
                manCostCount += 1
            else:
                break
                
        for i in range(12):
            if npWomanCost[i]<=minWomanCost[i]:
                womanCostCount += 1
            else:
                break
                
        if manCostCount==12:
            minManCost = npManCost
            minManTuningValue = data2[0]
            
        if womanCostCount==12:
            minWomanCost = npWomanCost
            minWomanTuningValue = data2[0]

pd.DataFrame([[minManCost, minManTuningValue],[minWomanCost,minWomanTuningValue]])
print(count)

6111369


In [3]:
print(minManCost)
print(minManTuningValue)
print(minWomanCost)
print(minWomanTuningValue)

NameError: name 'minManCost' is not defined

In [41]:




pd.DataFrame(makeLogP(range(11)))

,0
0,0.000000
1,0.398942
2,0.156874
3,0.072728
4,0.038153
5,0.021851
6,0.013355
7,0.008582
8,0.005739
9,0.003966


In [6]:
pd.DataFrame(idoctor.makeLogModel([20, 40, 0.5, 0.5, 0.5, 1]))

,0
0,0.000000e+00
1,0.000000e+00
2,0.000000e+00
3,0.000000e+00
4,0.000000e+00
5,0.000000e+00
6,0.000000e+00
7,0.000000e+00
8,0.000000e+00
9,0.000000e+00
